<a href="https://colab.research.google.com/github/sunweihao1997/swh_code/blob/master/Test_timeaxis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# if necessary, install NeuralGCM and dependencies

! pip install -q -U neuralgcm dinosaur gcsfs zarr==2.18.4

In [2]:
import jax
import numpy as np
import gcsfs
import neuralgcm
import xarray

# load demo model
checkpoint = neuralgcm.demo.load_checkpoint_tl63_stochastic()
model = neuralgcm.PressureLevelModel.from_checkpoint(checkpoint)

# create a xarray.Dataset with required variables for NeuralGCM
gcs = gcsfs.GCSFileSystem(token='anon')
path = 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'
full_era5 = xarray.open_zarr(gcs.get_mapper(path), chunks=None)
full_era5 = full_era5[model.input_variables + model.forcing_variables]

In [3]:
full_era5['u_component_of_wind'].data[0, 5, 50, 50]

<xarray.Dataset> Size: 1PB
Dimensions:                              (time: 1323648, level: 37,
                                          latitude: 721, longitude: 1440)
Coordinates:
  * latitude                             (latitude) float32 3kB 90.0 ... -90.0
  * level                                (level) int64 296B 1 2 3 ... 975 1000
  * longitude                            (longitude) float32 6kB 0.0 ... 359.8
  * time                                 (time) datetime64[ns] 11MB 1900-01-0...
Data variables:
    geopotential                         (time, level, latitude, longitude) float32 203TB ...
    specific_humidity                    (time, level, latitude, longitude) float32 203TB ...
    temperature                          (time, level, latitude, longitude) float32 203TB ...
    u_component_of_wind                  (time, level, latitude, longitude) float32 203TB ...
    v_component_of_wind                  (time, level, latitude, longitude) float32 203TB ...
    specific_cloud_ice_water_content     (time, level, latitude, longitude) float32 203TB ...
    specific_cloud_liquid_water_content  (time, level, latitude, longitude) float32 203TB ...
    sea_ice_cover                        (time, latitude, longitude) float32 5TB ...
    sea_surface_temperature              (time, latitude, longitude) float32 5TB ...
Attributes:
    valid_time_start:       1940-01-01
    last_updated:           2025-03-03 01:54:12.841944+00:00
    valid_time_stop:        2024-11-30
    valid_time_stop_era5t:  2025-02-25

In [31]:
select_era5 = full_era5.sel(time='2023-01-05')
select_era5['sea_surface_temperature'].data[:, 500, 1000]

array([295.5989, 295.5989, 295.5989, 295.5989, 295.5989, 295.5989,
       295.5989, 295.5989, 295.5989, 295.5989, 295.5989, 295.5989,
       295.5989, 295.5989, 295.5989, 295.5989, 295.5989, 295.5989,
       295.5989, 295.5989, 295.5989, 295.5989, 295.2627, 295.2627],
      dtype=float32)

In [32]:
from dinosaur import horizontal_interpolation
from dinosaur import spherical_harmonic
from dinosaur import xarray_utils


shift_era5 = xarray_utils.selective_temporal_shift(
    dataset=full_era5,
    variables=model.forcing_variables,
    time_shift='24 hours',
)

In [33]:
select_era5 = shift_era5.sel(time='2023-01-05')
select_era5['sea_surface_temperature'].data[:, 500, 1000]

array([295.75476, 295.75476, 295.75476, 295.75476, 295.75476, 295.75476,
       295.75476, 295.75476, 295.75476, 295.75476, 295.75476, 295.75476,
       295.75476, 295.75476, 295.75476, 295.75476, 295.75476, 295.75476,
       295.75476, 295.75476, 295.75476, 295.75476, 295.59885, 295.59885],
      dtype=float32)